## Import

In [48]:
import random
import pandas as pd
import numpy as np
import os
import cv2


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [49]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [50]:
CFG = {
    'IMG_SIZE':400,
    'EPOCHS':20,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [51]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

def get_currentTime():
    from datetime import date, datetime, timezone, timedelta

    exp_day = str(date.today())

    KST = timezone(timedelta(hours=9))
    time_record = datetime.now(KST)
    _day = str(time_record)[:10]
    _time = str(time_record.time())[:8]

    return _day

## Data Load

In [52]:
from operator import add
df = pd.read_csv('./train.csv')
bg_path = './COCOval17/'
bg_img_list = [bg_path+one for one in os.listdir(bg_path)]


## Train / Validation Split

In [53]:
df = df.sample(frac=1)
train_len = int(len(df) * 0.99)

In [54]:
train = df[:train_len]
val = df[train_len:]

## Data Preprocessing

In [55]:
def get_labels(df):
    return df.iloc[:,2:].values

In [56]:
train_labels = get_labels(train)
val_labels = get_labels(val)

## CustomDataset

In [57]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, bg_path_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.bg_path_list = bg_path_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        if self.bg_path_list is not None:
            bg_img = cv2.imread(random.choice(self.bg_path_list))
            bg_img = cv2.resize(bg_img,
                                (CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                                interpolation=cv2.INTER_CUBIC)


            fg_mask, bg_mask = self.get_mask(image)
            image = self.synthesis_imgs(image, bg_img, fg_mask, bg_mask)
        
        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=fg_mask)
            image = transformed['image']
            mask  = transformed['mask']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label, mask
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

    def get_mask(self, src_img):

        gray_img = cv2.cvtColor(src_img, cv2.COLOR_RGB2GRAY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        closing = cv2.morphologyEx(gray_img, cv2.MORPH_CLOSE, kernel)

        _, fg_mask = cv2.threshold(closing, 200, 1, cv2.THRESH_BINARY_INV)
        _, bg_mask = cv2.threshold(closing, 200, 1, cv2.THRESH_BINARY)

        return fg_mask, bg_mask

    def synthesis_imgs(self, fg_img, bg_img, fg_mask, bg_mask):
        
        masked_fg = fg_img * fg_mask[:, :, np.newaxis]
        masked_bg = bg_img * bg_mask[:, :, np.newaxis]

        return masked_fg + masked_bg



In [58]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            # A.RandomCrop(width=CFG['IMG_SIZE'], height=CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            # A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
                            # A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
                            # A.GaussNoise(var_limit=(0, 20)),
                            A.Rotate(limit=10, interpolation=1, border_mode=4, value=None, mask_value=None, rotate_method='largest_box', crop_border=False, always_apply=False, p=0.5),
                            A.RandomBrightnessContrast(p=0.2),
                            A.HorizontalFlip(),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            # A.RandomCrop(width=CFG['IMG_SIZE'], height=CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [59]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, bg_img_list, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val_labels, bg_img_list, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [60]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)

        # self.conv1.weight = nn.Parameter(self.backbone.classifier[1].weight.view(1000, 2048, 1, 1))
        # self.conv1.bias   = nn.Parameter(self.backbone.classifier[1].bias)

        # del self.backbone.classifier

        self.clf = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.clf(x))
        return x

## Train

In [61]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc
    
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels, masks in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}], val Loss : [{_val_loss: 0.5f}], val_acc : [{_val_acc: 0.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            # scheduler.step(epoch-1)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model

            now_day = get_currentTime()
            save_path = './archive/'
            torch.save(best_model.state_dict(), f'{save_path}{now_day}__train_99_epoch{epoch}.pt')

            
    
    return best_model

## Run!!

In [63]:
model = BaseModel()
model.load_state_dict(torch.load('E:/git/DACON/archive/2023-01-16__cosineLRsceduler_epoch56.pt', map_location=device))

<All keys matched successfully>

In [64]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.000001)
# CosineAnnealingWarmUpRestarts
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 11/11 [00:03<00:00,  2.78it/s]


Epoch [1], Train Loss : [0.26799], val Loss : [ 0.09863], val_acc : [ 0.96256]


100%|██████████| 11/11 [00:03<00:00,  2.79it/s]


Epoch [2], Train Loss : [0.11682], val Loss : [ 0.06481], val_acc : [ 0.97443]


100%|██████████| 11/11 [00:03<00:00,  2.78it/s]


...

100%|██████████| 11/11 [00:06<00:00,  1.63it/s]


Epoch [14], Train Loss : [0.01839], val Loss : [ 0.03321], val_acc : [ 0.98881]


100%|██████████| 11/11 [03:24<00:00, 18.60s/it]


Epoch [15], Train Loss : [0.01809], val Loss : [ 0.02960], val_acc : [ 0.99023]
Epoch 00015: reducing learning rate of group 0 to 7.5000e-05.


 89%|████████▉ | 913/1021 [8:43:29<01:42,  1.05it/s]    

## Inference

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [ ]:
preds = inference(model, test_loader, device)

100%|██████████| 46/46 [00:08<00:00,  5.19it/s]


## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit.iloc[:,1:] = preds
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,0,0,0,1,1,0,0,0,0
2,TEST_00002,1,1,0,0,1,1,0,1,0,1
3,TEST_00003,1,1,0,0,1,1,0,1,1,0
4,TEST_00004,0,1,0,0,1,0,0,0,0,0


In [ ]:
save_path = './prediction/'
exp_day = get_currentTime()
exp_label = 'effi_b0, (400, bgSyn, test False, epoch-20, norm first(basic A), train_99)'
extension = '.csv'
submit.to_csv(f'{save_path}{exp_day}__{exp_label}{extension}', index=False)